In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import pandas as pd
import json
import tensorflow as tf
import torch
import numpy as np


# PATH HERE
json_file = "/kaggle/input/kkkkkkkkkkkkkk/merged_data_10000_ours.json"

data = []
with open(json_file, 'r') as file:
    for line in file:
        data.append(json.loads(line))

train_df = pd.DataFrame(data)


In [ ]:
import os
import numpy as np

def concatenate_arrays(folder_path):
    arrays = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.npy'):
            array_path = os.path.join(folder_path, filename)
            array = np.load(array_path)
            arrays.append(array)
    return arrays


# PATH HERE
folder_path = '/kaggle/input/embeddingsdata/video_features_new/video_features_new'
result_array = concatenate_arrays(folder_path)
video_array = np.array(result_array)


In [ ]:
# PATH HERE
arr = np.load("/kaggle/input/embeddingsdata/video_features_new/video_features_new/00000002.npy")


In [ ]:
import os
import numpy as np

def concatenate_arrays(folder_path):
    arrays = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.npy'):
            array_path = os.path.join(folder_path, filename)
            array = np.load(array_path)
            arrays.append(array)
    return arrays


# PATH HERE
folder_patha = '/kaggle/input/audio-feats/Audio_features'
result_arraya = concatenate_arrays(folder_patha)
audio_array = np.array(result_arraya)

In [ ]:
# PATH HERE
hinglish_array = np.load("/kaggle/input/embeddingsdata/embeddings.npy")
print(hinglish_array.shape)

# PATH HERE
english_array = np.load("/kaggle/input/english-question-embeddings/english_question_embeddings.npy")
print(english_array.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df['labels'] = le.fit_transform(train_df['anser'])
train_df.head()

In [ ]:
train_df = train_df[['video_id', 'question_id', 'type', 'labels']]

In [ ]:
from sklearn.model_selection import train_test_split
X_trainv, X_testv, y_train, y_test = train_test_split(video_array, train_df['labels'], test_size=0.2, random_state=42)

print("For Video:")
print("X_train shape:", X_trainv.shape)
print("X_test shape:", X_testv.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_traina, X_testa, y_train, y_test = train_test_split(audio_array, train_df['labels'], test_size=0.2, random_state=42)

print("For Audio:")
print("X_train shape:", X_traina.shape)
print("X_test shape:", X_testa.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)




X_traint, X_testt, y_train, y_test = train_test_split(hinglish_array, train_df['labels'], test_size=0.2, random_state=42)

print("For Audio:")
print("X_train shape:", X_traint.shape)
print("X_test shape:", X_testt.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
import keras_nlp
encoder1 = keras_nlp.layers.TransformerEncoder(intermediate_dim=120, num_heads=4)
encoder2 = keras_nlp.layers.TransformerEncoder(intermediate_dim=120, num_heads=4)
encoder3 = keras_nlp.layers.TransformerEncoder(intermediate_dim=120, num_heads=4)

In [ ]:
import tensorflow as tf

def average_fusion_model_CNN():
    # Define input layers for each modality
    input_video = tf.keras.Input(shape=[768, 1])
    input_audio = tf.keras.Input(shape=[768, 1])
    input_text = tf.keras.Input(shape=[768, 1])

    # CNN layers for video
    video_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_video)
    video_cnn = tf.keras.layers.MaxPooling1D()(video_cnn)
    video_cnn = tf.keras.layers.Flatten()(video_cnn)

    # CNN layers for audio
    audio_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_audio)
    audio_cnn = tf.keras.layers.MaxPooling1D()(audio_cnn)
    audio_cnn = tf.keras.layers.Flatten()(audio_cnn)

    # CNN layers for text
    text_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_text)
    text_cnn = tf.keras.layers.MaxPooling1D()(text_cnn)
    text_cnn = tf.keras.layers.Flatten()(text_cnn)

    # Early fusion: average the CNN outputs from each modality
    averaged_embeddings = tf.keras.layers.Average()([video_cnn, audio_cnn, text_cnn])

    # Classifier layers
    x = tf.keras.layers.Dense(200, activation='relu')(averaged_embeddings)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(90, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(56, activation='relu')(x)
    output = tf.keras.layers.Dense(2, activation='softmax')(x)

    # Create and return the model
    model = tf.keras.Model(inputs=[input_video, input_audio, input_text], outputs=output)
    return model

# Instantiate the model
early_fusion_cnn_model = create_early_fusion_cnn_model()

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time

def compile_and_train_model(model, X_train, y_train, X_test, y_test, epochs=20, batch_size=16):
    # Define the optimizer
    lr = 1e-3
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    
    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    # Define callbacks
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                     factor=0.2,
                                                     patience=1,
                                                     verbose=1,
                                                     min_delta=1e-4,
                                                     min_lr=1e-15,
                                                     mode='max')
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                     min_delta=1e-4,
                                                     patience=70,
                                                     mode='max',
                                                     restore_best_weights=True,
                                                     verbose=1)
    
    callbacks = [earlystopping, reduce_lr]

    # Train the model
    start_time = time.time()
    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        batch_size=batch_size,
                        epochs=epochs,
                        callbacks=callbacks)
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    # Plot training history
    plot_history(history)

def plot_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()



In [ ]:
early_fusion_cnn_model = average_fusion_model_CNN()
compile_and_train_model(early_fusion_cnn_model, [X_trainv, X_traint, X_traina], y_train, [X_testv, X_testt, X_testa], y_test)
